In [1]:
import pandas as pd
import numpy as np

In [2]:
import sys
sys.path.append('../../code')

from utils import binarize_log_data, remove_ambiguous_row, clean_continuous

In [3]:
slice = pd.read_csv('../../data/raw/chembl30_slice.csv')

/tmp/ipykernel_68753/4181517134.py:1: DtypeWarning: Columns (10) have mixed types. Specify dtype option on import or set low_memory=False.
  slice = pd.read_csv('../../data/raw/chembl30_slice.csv')


In [4]:
slice['standard_value'] = slice['standard_value'].map(lambda value: -np.log10((max(value, 0) + 1)) + 9)
slice = slice[slice['standard_type'] == 'IC50']
kdr = slice[slice['gene_symbol'] == 'KDR']
kdr

,Unnamed: 0,compound_chembl_id,canonical_smiles,standard_type,standard_value,standard_units,standard_relation,assay_id,target_chembl_id,activity_comment,data_validity_comment,organism,pref_name,gene_symbol
368,6996,CHEMBL6246,O=c1oc2c(O)c(O)cc3c(=O)oc4c(O)c(O)cc1c4c23,IC50,6.101824,nM,=,808259,CHEMBL279,NaN,NaN,Homo sapiens,Vascular endothelial growth factor receptor 2,KDR
8128,113927,CHEMBL7724,Cc1cc2ncc(-c3ccccc3)nc2cc1C,IC50,5.207538,nM,=,603363,CHEMBL279,NaN,NaN,Homo sapiens,Vascular endothelial growth factor receptor 2,KDR
11424,183777,CHEMBL50,O=c1c(O)c(-c2ccc(O)c(O)c2)oc2cc(O)cc(O)c12,IC50,6.551294,nM,=,1284294,CHEMBL279,NaN,NaN,Homo sapiens,Vascular endothelial growth factor receptor 2,KDR
11480,184225,CHEMBL50,O=c1c(O)c(-c2ccc(O)c(O)c2)oc2cc(O)cc(O)c12,IC50,5.779630,nM,=,1774202,CHEMBL279,NaN,NaN,Homo sapiens,Vascular endothelial growth factor receptor 2,KDR
19701,350004,CHEMBL98,O=C(CCCCCCC(=O)Nc1ccccc1)NO,IC50,4.999957,nM,>,1527399,CHEMBL279,NaN,NaN,Homo sapiens,Vascular endothelial growth factor receptor 2,KDR
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2668327,16425523,CHEMBL4877264,CC(=O)N1CCN([C@H]2C[C@@H](n3cc(-c4ccc(Cl)c(N)c...,IC50,6.075204,nM,=,2125061,CHEMBL279,NaN,NaN,Homo sapiens,Vascular endothelial growth factor receptor 2,KDR
2668516,16426271,CHEMBL4877505,CCCNc1cc(Oc2ccc(NC(=O)Nc3cccc(C(F)(F)F)c3)cc2)...,IC50,7.698970,nM,=,2132287,CHEMBL279,NaN,NaN,Homo sapiens,Vascular endothelial growth factor receptor 2,KDR
2669245,16430243,CHEMBL4878552,Cc1ccc(NC(=O)Cn2cc(-c3ccc4c(NC(=O)c5cnn(C)c5)n...,IC50,8.761954,nM,=,2111788,CHEMBL279,NaN,NaN,Homo sapiens,Vascular endothelial growth factor receptor 2,KDR
2669269,16430348,CHEMBL4878575,O=C(CSc1nc2ccccc2n2cnnc12)Nc1ccc(C(=O)NC2CCCCC...,IC50,7.296709,nM,=,2102714,CHEMBL279,NaN,NaN,Homo sapiens,Vascular endothelial growth factor receptor 2,KDR


# Prepare KDR-Hi

In [5]:
kdr_binary = binarize_log_data(kdr, threshold=6)
kdr_binary = remove_ambiguous_row(kdr_binary)
kdr_hi = pd.DataFrame({
    'smiles': kdr_binary.index.to_list(),
    'value': kdr_binary['label'] > 0.5
}).reset_index(drop=True)
kdr_hi

,smiles,value
0,Brc1ccc(-c2nc3ccc(Nc4ccnc5ccccc45)cc3[nH]2)cc1,True
1,Brc1ccc(-c2nc3ccc(Nc4ncnc5ccccc45)cc3[nH]2)cc1,True
2,Brc1cccc(Nc2ncnc3ccc(NCc4ccc5c(c4)OCCCO5)cc23)c1,False
3,C#CC(C)OC1=CC(=O)C(Nc2ncnc3cc(OCCCN4CCCC4)c(OC...,True
4,C#CC(OC1=CC(=O)C(Nc2ncnc3cc(OCCCN4CCCC4)c(OC)c...,True
...,...,...
6272,c1cn(Cc2ccc3c(c2)-c2[nH]nc(-c4ccsc4)c2C3)cn1,True
6273,c1cnc(CCc2ccncc2)c(-c2nnc(NCc3ccc4c(c3)OCO4)o2)c1,True
6274,c1cnc(CCc2ccncc2)c(-c2nnc(Nc3cnc4ccccc4c3)o2)c1,True
6275,c1cncc(-c2cnn3cc(-c4ccc(OCCN5CCCCC5)cc4)cnc23)c1,True


In [6]:
kdr_hi.to_csv('../../data/raw/kdr_hi.csv')

# Prepare KDR-Lo

In [7]:
kdr_continuous = clean_continuous(kdr)
kdr_lo = pd.DataFrame({
    'smiles': kdr_continuous['canonical_smiles'],
    'value': kdr_continuous['standard_value']
}).reset_index(drop=True)
kdr_lo

/home/simon/papers/lohi/notebooks/data/../../code/utils.py:85: FutureWarning: Dropping invalid columns in DataFrameGroupBy.min is deprecated. In a future version, a TypeError will be raised. Before calling .min, select only columns which should be valid for the function.
  min_values = group.min()
/home/simon/papers/lohi/notebooks/data/../../code/utils.py:86: FutureWarning: Dropping invalid columns in DataFrameGroupBy.max is deprecated. In a future version, a TypeError will be raised. Before calling .max, select only columns which should be valid for the function.
  max_values = group.max()
/home/simon/papers/lohi/notebooks/data/../../code/utils.py:90: FutureWarning: The default value of numeric_only in DataFrameGroupBy.median is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  continuous_clean = group.median()


,smiles,value
0,Brc1ccc(-c2nc3ccc(Nc4ccnc5ccccc45)cc3[nH]2)cc1,6.742321
1,Brc1ccc(-c2nc3ccc(Nc4ncnc5ccccc45)cc3[nH]2)cc1,6.419075
2,C#CC(C)OC1=CC(=O)C(Nc2ncnc3cc(OCCCN4CCCC4)c(OC...,7.432974
3,C#CC(OC1=CC(=O)C(Nc2ncnc3cc(OCCCN4CCCC4)c(OC)c...,8.108191
4,C#CCO/N=C/c1c(N)ncnc1Oc1ccc2[nH]c(C)cc2c1F,7.275724
...,...,...
4890,c1cn(Cc2ccc3c(c2)-c2[nH]nc(-c4ccsc4)c2C3)cn1,6.680695
4891,c1cnc(CCc2ccncc2)c(-c2nnc(NCc3ccc4c(c3)OCO4)o2)c1,6.800804
4892,c1cnc(CCc2ccncc2)c(-c2nnc(Nc3cnc4ccccc4c3)o2)c1,6.419075
4893,c1cncc(-c2cnn3cc(-c4ccc(OCCN5CCCCC5)cc4)cnc23)c1,7.677781


In [8]:
kdr_lo.to_csv('../../data/raw/kdr_lo.csv')